***
## Table of Contents
<br>
  <br>
    <ol>
        <h2><a href='#1' style="text-decoration : none; color : #000000;">1. Initial Models</a></h2>
        <ul>
            <a href='#1.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#1.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br> 

<ol>
        <h2><a href='#2' style="text-decoration : none; color : #000000;">2. Oversampling </a></h2>
        <ul>
            <a href='#2.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#2.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br>
     
 <ol>
        <h2><a href='#3' style="text-decoration : none; color : #000000;">3. Undersampling</a></h2>
        <ul>
            <a href='#3.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#3.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br>
     
 <ol>
        <h2><a href='#4' style="text-decoration : none; color : #000000;">4.Feature Transformation with feature selection</a></h2>
        <ul>
            <a href='#4.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#4.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br>
     
     
 <ol>
        <h2><a href='#5' style="text-decoration : none; color : #000000;">5.Feature Transformation with PCA</a></h2>
        <ul>
            <a href='#5.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#5.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br>

In [27]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display,clear_output,HTML
import warnings
warnings.filterwarnings("ignore")

import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential
warnings.simplefilter("ignore", DeprecationWarning)
import researchpy as rp

%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',305)

from sklearn.feature_selection import SelectKBest,chi2,RFE,RFECV,f_regression,SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn import metrics



from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,auc,roc_curve,recall_score,precision_score,matthews_corrcoef

from pandas_ml import ConfusionMatrix


from imblearn.over_sampling import SMOTE

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import featuretools as ft

from scipy import stats
import re

In [28]:
### Function to plot Feature Importance
def feat_impt_plot(feat_impt,y_labels,width_s=1000,height_s=1000):
    
    m=pd.DataFrame(y_labels,feat_impt).reset_index()
    m.columns=['Feature_Importance','Features']
    m.sort_values(by='Feature_Importance',inplace=True,ascending=False)
    m['Features']=m['Features'].str.replace("dom_function_","")
    m['Features']=m['Features'].str.replace("js_function_","")
    m['Features']=m['Features'].apply(lambda x: str(x).lstrip('.'))
    m['Features']=m['Features'].str.replace("(","")
    
    data = [go.Bar(x=m.Feature_Importance.values,y=m.Features.values,text=np.round(m.Feature_Importance,4),
            textposition = 'outside',
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
            opacity=0.6,
            orientation='h'
        )]
    layout = go.Layout(autosize=False,
    width=width_s,
    height=height_s,
    xaxis=dict(title='Feature Importances',
        tickfont=dict(
            size=12,
            color='black'
        )),
    yaxis=dict(automargin=True))
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [29]:
def uni_value(data):
    lst=[]
    for col in data.columns[:]:
        lst.append([col,dict(data[col].value_counts()),data[col].value_counts().values[0]>=len(data)*0.95])
    col_values=pd.DataFrame(lst,columns=['Feature','Value_Counts','95_%_same_values'])
    col_values.set_index('Feature',drop=True,inplace=True)
    del lst
    return col_values

    
def check_cat(data):
    cat_var=[]
    num_var=[]
    for col in data.columns:
        if col!='Target':
            if len(data[col].value_counts())==2:
                cat_var.append(col)
            else:
                num_var.append(col)
    return cat_var,num_var


def num_stats(data,columns):
    lst=[]
    c=0
    for col in columns:
        corr=stats.pointbiserialr(data[col],data.Target)[0]
        corr_p=stats.pointbiserialr(data[col],data.Target)[1]
        ttest=stats.ttest_ind(data[data.Target==1][col], data[data.Target==0][col], equal_var=False)[0]
        ttest_p=stats.ttest_ind(data[data.Target==1][col], data[data.Target==0][col], equal_var=False)[1]
        lst.append((corr,corr_p,ttest,ttest_p,col))
    stats_data=pd.DataFrame(lst,columns=['Pt. Biserial Corr','Pt. Biserial Corr p-val','ttest','ttest p-val','Feature'])
    stats_data.set_index('Feature',drop=True,inplace=True)
    stats_data.sort_values(by='ttest',inplace=True)
    del lst
    return stats_data


### Generate statistics for categorical data
def cat_stats(data,columns):
    lst=[]
    c=0
    for col in columns:
        if col!='Target':
            table, results = rp.crosstab(data['Target'], data[col], prop= 'col', test= 'chi-square')
            x=results['results'].values.tolist()
            x.append(col)
            lst.append(x)
    chi_df=pd.DataFrame(lst,columns=['Pearson Chi-square','p-value','Cramers V','Feature'])
    chi_df.set_index('Feature',drop=True,inplace=True)
    chi_df.sort_values(by='Cramers V',ascending=False,inplace=True)
    del lst
    return chi_df

def remove_feat(data):
    col_no_unique=[]
    for col in data.columns[:]:
        if len(data[col].value_counts())<=1:
#             print(col)
            col_no_unique.append(col)
    return col_no_unique

### Generate plot for class balance for categorical nature
def class_bal(data,title=''):
    d = [go.Bar(x=data.Target.value_counts().index.values,
               y=data.Target.value_counts().values,
               text=data.Target.value_counts().values,
               textposition = 'outside',
            )]
    layout = go.Layout(title=title,xaxis=dict(title='Domain Type',tickfont=dict(size=12,color='black')),
                    yaxis=dict(title='Number of Instances'))
    fig = go.Figure(data=d, layout=layout)
    iplot(fig)

In [30]:
def model_build(model,X_train,Y_train,X_test,Y_test,tree=False,feat=None):
    
    y_pred_train=model.predict(X_train)
    acc_train=accuracy_score(Y_train,y_pred_train)
    
    print("Accuracy of the model for training data is:",acc_train)
    print("Confusion Matrix for training data is:")
    cm_train=ConfusionMatrix(Y_train,y_pred_train)
    display(cm_train)
    
    
    y_pred_test=model.predict(X_test)
    acc_test=accuracy_score(Y_test,y_pred_test)
    print("Accuracy of the model for test data is:",acc_test)
    print("Confusion Matrix for test data is:")
    cm_test=ConfusionMatrix(Y_test,y_pred_test)
    display(cm_test)

    fpr, tpr, threshold = roc_curve(Y_test, y_pred_test)
    roc_auc =auc(fpr, tpr)
    mcc=matthews_corrcoef(Y_test,y_pred_test)
    if tree==True:
        feat_impt_plot(model.feature_importances_,feat)
        return model,cm_train,cm_test,acc_train,acc_test,roc_auc,model.feature_importances_,mcc
    
    return model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc

def KNN_model(X_train,Y_train,X_test,Y_test):
    print('\033[1m' + 'K-Nearest Neighbors' + '\033[0m')
    knn=KNeighborsClassifier()
    knn.fit(X_train,Y_train)

    return model_build(knn,X_train,Y_train,X_test,Y_test)

def Logistic_model(X_train,Y_train,X_test,Y_test):
    print('\033[1m' + 'Logistic Regression' + '\033[0m')
    log=LogisticRegression(random_state=0)
    log.fit(X_train,Y_train)
    
    return model_build(log,X_train,Y_train,X_test,Y_test)

def RandomForest(X_train,Y_train,X_test,Y_test,feat):
    print('\033[1m' + 'RandomForest Classifier' + '\033[0m')
    rf=RandomForestClassifier(random_state=0)
    rf.fit(X_train,Y_train)

    
    return model_build(rf,X_train,Y_train,X_test,Y_test,True,feat)

def AdaBoost(X_train,Y_train,X_test,Y_test,feat):
    print('\033[1m' + 'Adaoost Classifier' + '\033[0m')
    ada=AdaBoostClassifier(random_state=0)
    ada.fit(X_train,Y_train)
    
    return model_build(ada,X_train,Y_train,X_test,Y_test,True,feat)

def GradientBoosting(X_train,Y_train,X_test,Y_test,feat):
    print('\033[1m' + 'GradientBoosting Classifier' + '\033[0m')
    gbc=GradientBoostingClassifier(random_state=0)
    gbc.fit(X_train,Y_train)
    
    return model_build(gbc,X_train,Y_train,X_test,Y_test,True,feat)

def ExtraTree(X_train,Y_train,X_test,Y_test,feat):
    print('\033[1m' + 'ExtraTree Classifier' + '\033[0m')
    ext_tree=ExtraTreesClassifier(random_state=0)
    ext_tree.fit(X_train,Y_train)
    
    return model_build(ext_tree,X_train,Y_train,X_test,Y_test,True,feat)

def Bagging(X_train,Y_train,X_test,Y_test):
    print('\033[1m' + 'Bagging Classifier' + '\033[0m')
    bagging=BaggingClassifier(random_state=0)
    bagging.fit(X_train,Y_train)
    
    return model_build(bagging,X_train,Y_train,X_test,Y_test)

def DeepLearning1(X_train,Y_train,X_test,Y_test):
    print('\033[1m' + 'Neural Network-1' + '\033[0m')
    np.random.seed(0)
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=X_train.shape[1]))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    deep_learning.fit(X_train,Y_train,batch_size=20,epochs=80,verbose=False)
    acc_train=deep_learning.evaluate(X_train,Y_train,verbose=False)[1]
    print("The accuracy of the model on training data is:",acc_train)
    cm_train=ConfusionMatrix(Y_train,deep_learning.predict_classes(X_train,batch_size=1,verbose=False).reshape(1,len(X_train))[0])
    cm_test=ConfusionMatrix(Y_test,deep_learning.predict_classes(X_test,batch_size=1,verbose=False).reshape(1,len(X_test))[0])
    display(cm_train)
    acc_test=accuracy_score(Y_test,deep_learning.predict_classes(X_test,batch_size=1,verbose=False))
    print("The accuracy of the model on test data is:",acc_test)
    display(cm_test)
    mcc=matthews_corrcoef(Y_test,deep_learning.predict_classes(X_test,batch_size=1,verbose=False))
    fpr, tpr, threshold = roc_curve(Y_test, deep_learning.predict_classes(X_test,batch_size=1,verbose=False))
    roc_auc =auc(fpr, tpr)
    return  deep_learning,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc

In [31]:
df=pd.read_csv('URL.csv')

df.drop(['Unnamed: 0'],axis=1,inplace=True)

df.fillna(value=0,inplace=True)

cookies=[columns for columns in df.columns if 'number' in columns]

df.drop(cookies,axis=1,inplace=True)
df=df[df['domain']!=0]
df.sort_values(by='Target',inplace=True,ascending=False)

df.drop_duplicates(['domain'], keep='first',inplace=True)

df=df.sample(frac=1,random_state=0)
print("Shape of our dataset:",df.shape)

Shape of our dataset: (46771, 4195)


In [32]:
df['url_number_vowels'] = df['domain'].str.lower().str.count(r'[aeiou]')
df['url_number_consonants'] = df['domain'].str.count(r'[bcdfghjklmnpqrstvwxzy]', flags=re.I)
df['url_number_digits'] = df['domain'].str.count(r'\d')
df['url_number_special_chars']=df['url_length']-df[['url_number_vowels','url_number_consonants','url_number_digits']].sum(axis=1)

In [33]:
df['url_port']=df['url_port'].apply(lambda x: 0 if x==0 else 1 )

In [34]:
# characters='abcdefghijklmnopqrstuvwxyz0123456789'
# for char in characters:
#     field=char+"_average"
#     df[field]=df['domain'].apply(lambda x: str.count(x, char))
#     df[field]=df[field]/df['url_length']

In [35]:
df.set_index(['domain'],inplace=True,drop=True)

In [36]:
data = [go.Bar(x=df.Target.value_counts().index.values,
               y=df.Target.value_counts().values,
               text=df.Target.value_counts().values,
               textposition = 'outside',
            )]
layout = go.Layout( xaxis=dict(title='Domain Type',tickfont=dict(size=12,color='black')),
                    yaxis=dict(title='Number of Instances'))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [37]:
col_no_unique=[]
for col in df.columns[:-1]:
    if len(df[col].value_counts())==1:
        col_no_unique.append(col)

print("{} columns have only one unique value in them rendering them meaningless for classification".format(len(col_no_unique)))
# display(np.array(col_no_unique))
### Drop those columns
df.drop(col_no_unique,axis=1,inplace=True)

302 columns have only one unique value in them rendering them meaningless for classification


In [38]:
print("Shape of our dataset after dropping them:",df.shape)

ext_end=[col for col in df.columns if 'extension_ends' in col]

for col in ext_end:
    df[col]=df[col].astype(int)

# ### Check and converting data types
# df['Target']=df.Target.apply(lambda x: 1 if x=='Malicious' else 0)
print("Columns which are of type object:\n",df.dtypes[(df.dtypes=='object')])

lst=[]
for col in df.columns[:-1]:
    lst.append([col,dict(df[col].value_counts()),df[col].value_counts().values[0]>=len(df)*0.95])

col_values=pd.DataFrame(lst,columns=['Feature','Value_Counts','95_%_same_values'])
col_values.set_index('Feature',drop=True,inplace=True)

Shape of our dataset after dropping them: (46771, 3896)
Columns which are of type object:
 url_contains_signin    object
url_contains_www       object
url_host               object
url_ip                 object
dtype: object


In [39]:
print("{} columns have more than 95% same values".format(len(col_values[col_values['95_%_same_values']==True].index.values)))

columns=col_values[col_values['95_%_same_values']==False].index.values
columns=columns.tolist()
col_values['95_%_same_values'].value_counts()

3846 columns have more than 95% same values


True     3846
False      49
Name: 95_%_same_values, dtype: int64

In [40]:
print("Columns which are of type object:",df[columns].dtypes[(df.dtypes=='object')])
columns.remove('url_host')

rem=['url_tld_NET','url_tld_COM','url_tld_NE','url_tld_CO','url_extension_endswith_.com', 'url_extension_.c']
for col in rem:
    columns.remove(col)

df_sel=df[columns]

cat_var_js,num_var_js=check_cat(df_sel)

print("Stats for numerical features for HTML experiment\n")
num_stats_js=num_stats(df_sel,num_var_js)
display(num_stats_js)
print("\nStats for Categorical features for HTML experiment\n")
cat_stats_js=cat_stats(df_sel,cat_var_js)
display(cat_stats_js)

Columns which are of type object: url_host    object
dtype: object
Stats for numerical features for HTML experiment



Pt. Biserial Corr  Pt. Biserial Corr p-val  \
Feature                                                               
url_extension_.com               -0.154316            4.061880e-247   
url_extension_.i                 -0.005769             2.122020e-01   
url_char_m                       -0.000765             8.685861e-01   
url_char_o                        0.074650             8.715169e-59   
url_char_j                        0.116675            1.984745e-141   
url_char_v                        0.105337            1.682865e-115   
url_char_c                        0.110988            4.385017e-128   
url_char_g                        0.119543            2.045127e-148   
url_char_-                        0.188291             0.000000e+00   
url_char_k                        0.145710            2.890403e-220   
url_char_r                        0.140080            1.445021e-203   
url_char_d                        0.185027             0.000000e+00   
url_words_with_length_8           0.184637             0.000000e+00   
url_char_p                        0.187333             0.000000e+00   
url_char_h                        0.178102             0.000000e+00   
url_words_with_length_5           0.173285            5.034966e-312   
url_char_f                        0.189686             0.000000e+00   
url_char_t                        0.185356             0.000000e+00   
url_number_digits                 0.303942             0.000000e+00   
url_words_with_length_6           0.184873             0.000000e+00   
url_char_a                        0.180943             0.000000e+00   
url_char_b                        0.197619             0.000000e+00   
url_char_x                        0.224483             0.000000e+00   
url_char_u                        0.190467             0.000000e+00   
url_words_with_length_7           0.216715             0.000000e+00   
url_char_n                        0.213296             0.000000e+00   
url_tlds                          0.234182             0.000000e+00   
url_char_l                        0.234193             0.000000e+00   
url_char_s                        0.224374             0.000000e+00   
url_char_e                        0.235855             0.000000e+00   
url_char_y                        0.292354             0.000000e+00   
url_char_z                        0.306922             0.000000e+00   
url_words_with_length_4           0.260214             0.000000e+00   
url_number_consonants             0.404512             0.000000e+00   
url_number_vowels                 0.365966             0.000000e+00   
url_char_i                        0.287576             0.000000e+00   
url_length                        0.461816             0.000000e+00   
url_char_w                        0.429855             0.000000e+00   
url_extensions                    0.489530             0.000000e+00   
url_char_.                        0.527821             0.000000e+00   

                             ttest    ttest p-val  
Feature                                            
url_extension_.com      -34.113589  9.926271e-241  
url_extension_.i         -1.243880   2.135748e-01  
url_char_m               -0.133716   8.936303e-01  
url_char_o               12.338724   1.155801e-34  
url_char_j               16.725416   1.082992e-61  
url_char_v               17.027650   6.949488e-64  
url_char_c               17.136575   1.205077e-64  
url_char_g               20.120991   6.796065e-88  
url_char_-               22.532080  1.071961e-108  
url_char_k               22.710567  1.292852e-110  
url_char_r               24.001861  4.798345e-123  
url_char_d               26.310656  4.906239e-146  
url_words_with_length_8  26.632659  1.862759e-149  
url_char_p               27.120052  1.155928e-154  
url_char_h               27.362729  1.982512e-157  
url_words_with_length_5  28.602282  2.333533e-171  
url_char_f               28.688053  4.886014e-172  
url_char_t               29.034757  4.458870e-176  
ur


Stats for Categorical features for HTML experiment



Pearson Chi-square  p-value  Cramers V
Feature                                                   
url_extension_.net            138.8931      0.0     0.0545

In [41]:
fin_col=columns
fin_col

['Target',
 'url_char_-',
 'url_char_.',
 'url_char_a',
 'url_char_b',
 'url_char_c',
 'url_char_d',
 'url_char_e',
 'url_char_f',
 'url_char_g',
 'url_char_h',
 'url_char_i',
 'url_char_j',
 'url_char_k',
 'url_char_l',
 'url_char_m',
 'url_char_n',
 'url_char_o',
 'url_char_p',
 'url_char_r',
 'url_char_s',
 'url_char_t',
 'url_char_u',
 'url_char_v',
 'url_char_w',
 'url_char_x',
 'url_char_y',
 'url_char_z',
 'url_extension_.com',
 'url_extension_.i',
 'url_extension_.net',
 'url_extensions',
 'url_length',
 'url_tlds',
 'url_words_with_length_4',
 'url_words_with_length_5',
 'url_words_with_length_6',
 'url_words_with_length_7',
 'url_words_with_length_8',
 'url_number_vowels',
 'url_number_consonants',
 'url_number_digits']

In [42]:
train=df_sel.iloc[:37417,:]
train_X=train.iloc[:,train.columns!='Target'].values
train_Y=train.Target.values

scaler=StandardScaler()
scaled_X_train=scaler.fit_transform(train_X)

test=df_sel.iloc[37417:,:]
test_X=test.iloc[:,test.columns!='Target'].values
test_Y=test.Target.values
scaled_X_test=scaler.transform(test_X)

In [43]:
pd.Series(df_sel[df_sel.Target==0].index.values).to_csv('benign_URL.csv')
pd.Series(df_sel[df_sel.Target==1].index.values).to_csv('malicious_URL.csv')

In [44]:
columns=fin_col

### <a id='1' style="text-decoration : none; color : #000000;"> 2. Initial Models</a>

In [45]:
columns.remove('Target')

In [46]:
ind_columns=['KNeighborsClassifier','LogisticRegression','RandomForestClassifier','AdaBoostClassifier',
             'GradientBoostingClassifier','ExtraTreeClassifier','BaggingClassifier','Neural Network-1']

f=[KNN_model,Logistic_model,RandomForest,AdaBoost,GradientBoosting, ExtraTree,Bagging,DeepLearning1]

## Table to store training and test measures
table_80_50_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_50_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','MCC'])

In [47]:
feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](scaled_X_train,train_Y,scaled_X_test,test_Y,columns)
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](scaled_X_train,train_Y,scaled_X_test,test_Y)
    table_80_50_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_50_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.9645348371061282
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False      31827    65    31892
True        1262  4263     5525
__all__    33089  4328    37417

Accuracy of the model for test data is: 0.9584135129356425
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       7959    26     7985
True         363  1006     1369
__all__     8322  1032     9354



Logistic Regression
Accuracy of the model for training data is: 0.9516529919555282
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       7904    81     7985
True         281  1088     1369
__all__     8185  1169     9354



Neural Network-1
The accuracy of the model on training data is: 0.9959376753881926


Predicted  False  True  __all__
Actual                         
False      31851    41    31892
True         111  5414     5525
__all__    31962  5455    37417

The accuracy of the model on test data is: 0.9557408595253367


Predicted  False  True  __all__
Actual                         
False       7788   197     7985
True         217  1152     1369
__all__     8005  1349     9354

### <a id='1.A' style="text-decoration : none; color : #000000;"> A Results</a>

In [48]:
trace0 = go.Scatter(x=table_80_50_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_50_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_50_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_50_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_50_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_50_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_50_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_50_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [49]:
table_80_50_train['index']=ind_columns
table_80_50_test['index']=ind_columns


table_80_50_test.set_index(['index'],drop=True,inplace=True)
table_80_50_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_50_train)
print("Test Results")
display(table_80_50_test)

Training Results


accuracy      TP       TN     FP      FN  \
index                                                                  
KNeighborsClassifier        0.964535  4263.0  31827.0   65.0  1262.0   
LogisticRegression          0.951653  4117.0  31491.0  401.0  1408.0   
RandomForestClassifier      0.994521  5323.0  31889.0    3.0   202.0   
AdaBoostClassifier          0.948793  4053.0  31448.0  444.0  1472.0   
GradientBoostingClassifier  0.959831  4362.0  31552.0  340.0  1163.0   
ExtraTreeClassifier         0.999973  5524.0  31892.0    0.0     1.0   
BaggingClassifier           0.994200  5315.0  31885.0    7.0   210.0   
Neural Network-1            0.995938  5414.0  31851.0   41.0   111.0   

                                 FPR       FNR  
index                                           
KNeighborsClassifier        0.002038  0.228416  
LogisticRegression          0.012574  0.254842  
RandomForestClassifier      0.000094  0.036561  
AdaBoostClassifier          0.013922  0.266425  
GradientBoostingClassifier  0.010661  0.210498  
ExtraTreeClassifier         0.000000  0.000181  
BaggingClassifier           0.000219  0.038009  
Neural Network-1            0.001286  0.020090

Test Results


accuracy      TP      TN     FP     FN       FPR  \
index                                                                          
KNeighborsClassifier        0.958414  1006.0  7959.0   26.0  363.0  0.003256   
LogisticRegression          0.951785  1033.0  7870.0  115.0  336.0  0.014402   
RandomForestClassifier      0.964614  1096.0  7927.0   58.0  273.0  0.007264   
AdaBoostClassifier          0.949861  1018.0  7867.0  118.0  351.0  0.014778   
GradientBoostingClassifier  0.957238  1068.0  7886.0   99.0  301.0  0.012398   
ExtraTreeClassifier         0.964080  1092.0  7926.0   59.0  277.0  0.007389   
BaggingClassifier           0.961300  1088.0  7904.0   81.0  281.0  0.010144   
Neural Network-1            0.955741  1152.0  7788.0  197.0  217.0  0.024671   

                                 FNR       auc       MCC  
index                                                     
KNeighborsClassifier        0.265157  0.865793  0.825375  
LogisticRegression          0.245435  0.870082  0.797315  
RandomForestClassifier      0.199416  0.896660  0.852664  
AdaBoostClassifier          0.256392  0.864415  0.788667  
GradientBoostingClassifier  0.219869  0.883867  0.821205  
ExtraTreeClassifier         0.202337  0.895137  0.850340  
BaggingClassifier           0.205259  0.892298  0.838627  
Neural Network-1            0.158510  0.908409  0.821823

### <a id='1.B' style="text-decoration : none; color : #000000;"> B. Average Feature Importances</a>

In [50]:
feat_impt_plot(np.average(feat_imp,axis=0),columns)

### <a id='2' style="text-decoration : none; color : #000000;"> 2. OverSampling </a>

In [51]:
sm = SMOTE(random_state=12, ratio = 1.0)
X_train, Y_train = sm.fit_sample(scaled_X_train,train_Y)

In [52]:
## Table to store training and test measures so we can compare later
table_80_80_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_80_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','mcc'])

In [53]:
feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](X_train,Y_train,scaled_X_test,test_Y,columns)
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](X_train,Y_train,scaled_X_test,test_Y)
    table_80_80_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_80_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.9760911827417534
Confusion Matrix for training data is:


Predicted  False   True  __all__
Actual                          
False      30398   1494    31892
True          31  31861    31892
__all__    30429  33355    63784

Accuracy of the model for test data is: 0.9173615565533462
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       7398   587     7985
True         186  1183     1369
__all__     7584  1770     9354



Logistic Regression
Accuracy of the model for training data is: 0.8986736485639032
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       7903    82     7985
True         272  1097     1369
__all__     8175  1179     9354



Neural Network-1
The accuracy of the model on training data is: 0.9979932271416029


Predicted  False   True  __all__
Actual                          
False      31789    103    31892
True          25  31867    31892
__all__    31814  31970    63784

The accuracy of the model on test data is: 0.9419499679281591


Predicted  False  True  __all__
Actual                         
False       7668   317     7985
True         226  1143     1369
__all__     7894  1460     9354

### <a id='2.A' style="text-decoration : none; color : #000000;"> A Results</a>

In [54]:
trace0 = go.Scatter(x=table_80_80_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_80_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_80_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_80_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_80_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_80_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_80_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_80_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [55]:
table_80_80_train['index']=ind_columns
table_80_80_test['index']=ind_columns


table_80_80_test.set_index(['index'],drop=True,inplace=True)
table_80_80_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_80_train)
print("Test Results")
display(table_80_80_test)

Training Results


accuracy       TP       TN      FP      FN  \
index                                                                    
KNeighborsClassifier        0.976091  31861.0  30398.0  1494.0    31.0   
LogisticRegression          0.898674  27685.0  29636.0  2256.0  4207.0   
RandomForestClassifier      0.997868  31762.0  31886.0     6.0   130.0   
AdaBoostClassifier          0.940283  29489.0  30486.0  1406.0  2403.0   
GradientBoostingClassifier  0.965305  30479.0  31092.0   800.0  1413.0   
ExtraTreeClassifier         0.999984  31891.0  31892.0     0.0     1.0   
BaggingClassifier           0.997272  31728.0  31882.0    10.0   164.0   
Neural Network-1            0.997993  31867.0  31789.0   103.0    25.0   

                                 FPR       FNR  
index                                           
KNeighborsClassifier        0.046846  0.000972  
LogisticRegression          0.070739  0.131914  
RandomForestClassifier      0.000188  0.004076  
AdaBoostClassifier          0.044086  0.075348  
GradientBoostingClassifier  0.025085  0.044306  
ExtraTreeClassifier         0.000000  0.000031  
BaggingClassifier           0.000314  0.005142  
Neural Network-1            0.003230  0.000784

Test Results


accuracy      TP      TN     FP     FN       FPR  \
index                                                                          
KNeighborsClassifier        0.917362  1183.0  7398.0  587.0  186.0  0.073513   
LogisticRegression          0.922707  1209.0  7422.0  563.0  160.0  0.070507   
RandomForestClassifier      0.962262  1105.0  7896.0   89.0  264.0  0.011146   
AdaBoostClassifier          0.932222  1115.0  7605.0  380.0  254.0  0.047589   
GradientBoostingClassifier  0.948578  1111.0  7762.0  223.0  258.0  0.027927   
ExtraTreeClassifier         0.964293  1105.0  7915.0   70.0  264.0  0.008766   
BaggingClassifier           0.962155  1097.0  7903.0   82.0  272.0  0.010269   
Neural Network-1            0.941950  1143.0  7668.0  317.0  226.0  0.039699   

                                 FNR       auc       mcc  
index                                                     
KNeighborsClassifier        0.135866  0.895311  0.713464  
LogisticRegression          0.116874  0.906310  0.732998  
RandomForestClassifier      0.192841  0.898006  0.843164  
AdaBoostClassifier          0.185537  0.883437  0.739706  
GradientBoostingClassifier  0.188459  0.891807  0.792093  
ExtraTreeClassifier         0.192841  0.899196  0.851503  
BaggingClassifier           0.198685  0.895523  0.842442  
Neural Network-1            0.165084  0.897608  0.774461

### <a id='2.B' style="text-decoration : none; color : #000000;"> B. Average Feature Importances</a>

In [56]:
feat_impt_plot(np.average(feat_imp,axis=0),columns)

### <a id='3' style="text-decoration : none; color : #000000;"> 3. Undersampling</a>

In [57]:
df_non=train[train.Target==0]
df_mal=train[train.Target==1]

df_non=df_non.sample(frac=len(df_mal)/len(df_non),random_state=0)
df_u=df_non.append(df_mal)
df_u.Target.value_counts() 

X_train=df_u[columns]
X_train=scaler.transform(X_train)
Y_train=df_u.Target.values

print("Shape after undersampling",X_train.shape)


Shape after undersampling (11050, 41)


In [58]:
## Table to store training and test measures so we can compare later
table_80_90_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_90_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','mcc'])

In [59]:
feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](X_train,Y_train,scaled_X_test,test_Y,columns)
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](X_train,Y_train,scaled_X_test,test_Y)
    table_80_90_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_90_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.9197285067873303
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False       5421   104     5525
True         783  4742     5525
__all__     6204  4846    11050

Accuracy of the model for test data is: 0.9468676502031217
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       7733   252     7985
True         245  1124     1369
__all__     7978  1376     9354



Logistic Regression
Accuracy of the model for training data is: 0.9009049773755656
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       7543   442     7985
True         180  1189     1369
__all__     7723  1631     9354



Neural Network-1
The accuracy of the model on training data is: 0.9979185520361991


Predicted  False  True  __all__
Actual                         
False       5525     0     5525
True          23  5502     5525
__all__     5548  5502    11050

The accuracy of the model on test data is: 0.9049604447295275


Predicted  False  True  __all__
Actual                         
False       7259   726     7985
True         163  1206     1369
__all__     7422  1932     9354

### <a id='3.A' style="text-decoration : none; color : #000000;"> A. Results</a>

In [60]:
trace0 = go.Scatter(x=table_80_90_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_90_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_90_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_90_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_90_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_90_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_90_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_90_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [61]:
table_80_90_train['index']=ind_columns
table_80_90_test['index']=ind_columns


table_80_90_test.set_index(['index'],drop=True,inplace=True)
table_80_90_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_90_train)
print("Test Results")
display(table_80_90_test)

Training Results


accuracy      TP      TN     FP     FN       FPR  \
index                                                                          
KNeighborsClassifier        0.919729  4742.0  5421.0  104.0  783.0  0.018824   
LogisticRegression          0.900905  4797.0  5158.0  367.0  728.0  0.066425   
RandomForestClassifier      0.990407  5422.0  5522.0    3.0  103.0  0.000543   
AdaBoostClassifier          0.897014  4825.0  5087.0  438.0  700.0  0.079276   
GradientBoostingClassifier  0.909593  4808.0  5243.0  282.0  717.0  0.051041   
ExtraTreeClassifier         0.999910  5524.0  5525.0    0.0    1.0  0.000000   
BaggingClassifier           0.989412  5417.0  5516.0    9.0  108.0  0.001629   
Neural Network-1            0.997919  5502.0  5525.0    0.0   23.0  0.000000   

                                 FNR  
index                                 
KNeighborsClassifier        0.141719  
LogisticRegression          0.131765  
RandomForestClassifier      0.018643  
AdaBoostClassifier          0.126697  
GradientBoostingClassifier  0.129774  
ExtraTreeClassifier         0.000181  
BaggingClassifier           0.019548  
Neural Network-1            0.004163

Test Results


accuracy      TP      TN     FP     FN       FPR  \
index                                                                          
KNeighborsClassifier        0.946868  1124.0  7733.0  252.0  245.0  0.031559   
LogisticRegression          0.919393  1199.0  7401.0  584.0  170.0  0.073137   
RandomForestClassifier      0.938743  1192.0  7589.0  396.0  177.0  0.049593   
AdaBoostClassifier          0.903250  1201.0  7248.0  737.0  168.0  0.092298   
GradientBoostingClassifier  0.931580  1196.0  7518.0  467.0  173.0  0.058485   
ExtraTreeClassifier         0.939277  1201.0  7585.0  400.0  168.0  0.050094   
BaggingClassifier           0.933504  1189.0  7543.0  442.0  180.0  0.055354   
Neural Network-1            0.904960  1206.0  7259.0  726.0  163.0  0.090920   

                                 FNR       auc       mcc  
index                                                     
KNeighborsClassifier        0.178963  0.894739  0.787813  
LogisticRegression          0.124178  0.901342  0.722324  
RandomForestClassifier      0.129291  0.910558  0.773071  
AdaBoostClassifier          0.122717  0.892492  0.684603  
GradientBoostingClassifier  0.126370  0.907573  0.753593  
ExtraTreeClassifier         0.122717  0.913594  0.776271  
BaggingClassifier           0.131483  0.906582  0.757525  
Neural Network-1            0.119065  0.895007  0.689779

### <a id='3.B' style="text-decoration : none; color : #000000;"> B. Average Feature Importances</a>

In [62]:
feat_impt_plot(np.average(feat_imp,axis=0),columns)

### <a id='4' style="text-decoration : none; color : #000000;">4. Feature Transformation with feature selection </a>

In [63]:
df.reset_index(inplace=True)

# columns=columns.tolist()
columns.append('domain')

df_min=df[columns]

In [64]:
es = ft.EntitySet(id = 'malicious')
es.entity_from_dataframe(entity_id = 'data', dataframe = df_min, index = 'domain')

feature_matrix_sessions, features_defs = ft.dfs(entityset=es,target_entity="data",
                                                trans_primitives = ['add','divide','multiply'],n_jobs=-1,
                                                verbose=1,max_depth=1)

Built 3321 features
EntitySet scattered to workers in 5.153 seconds
Elapsed: 00:23 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [65]:
df.set_index(['domain'],drop=True,inplace=True)
fs=pd.merge(feature_matrix_sessions,pd.DataFrame(df.Target),left_index=True, right_index=True)
fs.replace([np.inf],0,inplace=True)
fs.fillna(value=0,inplace=True)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/client.py", line 1223, in _close
    quiet_exceptions=(CancelledError,),
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 584, in with_timeout
    chain_future(future_converted, result)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 166, in chain_future
    future_add_done_callback(a, copy)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 262, in future_add_done_callback
    callback(future)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 160, in copy
    elif a.ex

In [66]:
X=fs.loc[:,fs.columns!='Target']
y=fs.Target
feature_name = X.columns.tolist()

In [67]:
def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-100:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature


cor_support, cor_feature = cor_selector(X, y)

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/client.py", line 1223, in _close
    quiet_exceptions=(CancelledError,),
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 584, in with_timeout
    chain_future(future_converted, result)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 166, in chain_future
    future_add_done_callback(a, copy)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 262, in future_add_done_callback
    callback(future)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 160, in copy
    elif a.exception() is not None:
concurrent.futures._base.CancelledError
Exception ignored in: <bound method

In [68]:
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=100)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()

rfe_selector = RFE(estimator=LogisticRegression(random_state=0), n_features_to_select=100, step=0.3, verbose=False)
rfe_selector.fit(X_norm, y)
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100,random_state=0), threshold='23*median')
embeded_rf_selector.fit(X, y)
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()

feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support,
                                    'Random Forest':embeded_rf_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)

In [69]:
X=fs[feature_selection_df[feature_selection_df.Total>=3]['Feature']]
X.fillna(value=0,inplace=True)
y=fs.Target.values
feature_name = X.columns.tolist()

scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

print("The train/test split ratio is 80:20")
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,y,random_state=0,test_size=0.2)

X.shape

The train/test split ratio is 80:20


(46771, 33)

In [70]:
print('Features we will be using:\n')
display(feature_name)

Features we will be using:



['url_char_w * url_extensions',
 'url_length + url_words_with_length_7',
 'url_length + url_tlds',
 'url_extension_.com / url_char_.',
 'url_extension_.com + url_length',
 'url_char_y + url_length',
 'url_char_w + url_length',
 'url_char_w + url_extensions',
 'url_char_w + url_char_z',
 'url_char_v + url_length',
 'url_char_u + url_length',
 'url_char_s + url_length',
 'url_char_r + url_length',
 'url_char_o + url_length',
 'url_char_m + url_length',
 'url_char_l + url_length',
 'url_char_k + url_length',
 'url_char_k + url_extensions',
 'url_char_j + url_length',
 'url_char_i + url_length',
 'url_char_h + url_length',
 'url_char_g + url_length',
 'url_char_f + url_length',
 'url_char_e + url_length',
 'url_char_c + url_length',
 'url_char_b + url_length',
 'url_char_a + url_length',
 'url_char_. + url_length',
 'url_char_. + url_char_z',
 'url_char_. + url_char_w',
 'url_char_. + url_char_j',
 'url_char_.',
 'url_char_- + url_length']

In [71]:
fs=fs.reindex(df_sel.index.values)

In [72]:
train=fs.iloc[:37417]
train_Y=train.Target.values
train_X=train[feature_name].values


scaler=StandardScaler()
scaled_X_train=scaler.fit_transform(train_X)

test=fs.iloc[37417:]
test_Y=test.Target.values
test_X=test[feature_name]

scaled_X_test=scaler.transform(test_X)

In [73]:
## Table to store training and test measures so we can compare later
table_80_60_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_60_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','mcc'])

In [74]:
feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](scaled_X_train,train_Y,scaled_X_test,test_Y,feature_name)
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](scaled_X_train,train_Y,scaled_X_test,test_Y)
    table_80_60_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_60_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.961301012908571
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False      31556   336    31892
True        1112  4413     5525
__all__    32668  4749    37417

Accuracy of the model for test data is: 0.953068206115031
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       7850   135     7985
True         304  1065     1369
__all__     8154  1200     9354



Logistic Regression
Accuracy of the model for training data is: 0.9427265681374776
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       7867   118     7985
True         307  1062     1369
__all__     8174  1180     9354



Neural Network-1
The accuracy of the model on training data is: 0.967180693267766


Predicted  False  True  __all__
Actual                         
False      31743   149    31892
True        1079  4446     5525
__all__    32822  4595    37417

The accuracy of the model on test data is: 0.9583066067992303


Predicted  False  True  __all__
Actual                         
False       7902    83     7985
True         307  1062     1369
__all__     8209  1145     9354

### <a id='4.A' style="text-decoration : none; color : #000000;"> A. Results</a>

In [75]:
trace0 = go.Scatter(x=table_80_60_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_60_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_60_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_60_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_60_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_60_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_60_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_60_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [76]:
table_80_60_train['index']=ind_columns
table_80_60_test['index']=ind_columns


table_80_60_test.set_index(['index'],drop=True,inplace=True)
table_80_60_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_60_train)
print("Test Results")
display(table_80_60_test)

Training Results


accuracy      TP       TN     FP      FN  \
index                                                                  
KNeighborsClassifier        0.961301  4413.0  31556.0  336.0  1112.0   
LogisticRegression          0.942727  3861.0  31413.0  479.0  1664.0   
RandomForestClassifier      0.993212  5285.0  31878.0   14.0   240.0   
AdaBoostClassifier          0.940482  3796.0  31394.0  498.0  1729.0   
GradientBoostingClassifier  0.951012  4171.0  31413.0  479.0  1354.0   
ExtraTreeClassifier         0.999786  5517.0  31892.0    0.0     8.0   
BaggingClassifier           0.993479  5292.0  31881.0   11.0   233.0   
Neural Network-1            0.967181  4446.0  31743.0  149.0  1079.0   

                                 FPR       FNR  
index                                           
KNeighborsClassifier        0.010536  0.201267  
LogisticRegression          0.015019  0.301176  
RandomForestClassifier      0.000439  0.043439  
AdaBoostClassifier          0.015615  0.312941  
GradientBoostingClassifier  0.015019  0.245068  
ExtraTreeClassifier         0.000000  0.001448  
BaggingClassifier           0.000345  0.042172  
Neural Network-1            0.004672  0.195294

Test Results


accuracy      TP      TN     FP     FN       FPR  \
index                                                                          
KNeighborsClassifier        0.953068  1065.0  7850.0  135.0  304.0  0.016907   
LogisticRegression          0.943767   974.0  7854.0  131.0  395.0  0.016406   
RandomForestClassifier      0.954565  1054.0  7875.0  110.0  315.0  0.013776   
AdaBoostClassifier          0.939919   943.0  7849.0  136.0  426.0  0.017032   
GradientBoostingClassifier  0.949754  1027.0  7857.0  128.0  342.0  0.016030   
ExtraTreeClassifier         0.953175  1059.0  7857.0  128.0  310.0  0.016030   
BaggingClassifier           0.954565  1062.0  7867.0  118.0  307.0  0.014778   
Neural Network-1            0.958307  1062.0  7902.0   83.0  307.0  0.010394   

                                 FNR       auc       mcc  
index                                                     
KNeighborsClassifier        0.222060  0.880517  0.804390  
LogisticRegression          0.288532  0.847531  0.761169  
RandomForestClassifier      0.230095  0.878065  0.809686  
AdaBoostClassifier          0.311176  0.835896  0.743326  
GradientBoostingClassifier  0.249817  0.867076  0.788777  
ExtraTreeClassifier         0.226443  0.878764  0.804416  
BaggingClassifier           0.224251  0.880486  0.810119  
Neural Network-1            0.224251  0.882677  0.825380

### <a id='4.B' style="text-decoration : none; color : #000000;"> B. Average Feature Importances</a>

In [77]:
feat_impt_plot(np.average(feat_imp,axis=0),feature_name)

### <a id='5' style="text-decoration : none; color : #000000;">5. Feature Transformation with PCA </a>

In [78]:
X=fs.loc[:,fs.columns!='Target']
y=fs.Target.values
feature_name = X.columns.tolist()
X=np.nan_to_num(X)
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

In [85]:
sns.set_style('whitegrid')
pca = PCA(random_state=0)
pca.fit(scaled_X)
plt.figure(1, figsize=(10, 8))
plt.plot(np.cumsum(pca.explained_variance_ratio_),'--o', linewidth=2)
plt.axis('tight')
plt.xlabel('Number of Components',size=14)
plt.xticks(size=12)
plt.xlim([0,200])
plt.ylabel('% Explained Variance',size=14)
plt.yticks(size=12)
plt.title('% Variance Explained by Principal Components',size=14,y=1.05)
plt.show()

distributed.core - INFO - Event loop was unresponsive in Nanny for 41.46s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 41.47s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 41.47s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 41.46s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 41.46s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. T

In [86]:
pca=PCA(n_components=110,whiten=True,random_state=0)
X_pca_21=pca.fit_transform(scaled_X)
print("Variance explained using 110 components is:",sum(pca.explained_variance_ratio_))

Variance explained using 110 components is: 0.8065851863478841


In [87]:
X_train=X_pca_21[:37417]
Y_train=fs.iloc[:37417]['Target'].values
X_test=X_pca_21[37417:]
Y_test=fs.iloc[37417:]['Target'].values

## Table to store training and test measures so we can compare later
table_80_70_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_70_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','mcc'])

In [88]:
feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](X_train,Y_train,X_test,Y_test,np.arange(0,110).astype(str))
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](X_train,Y_train,X_test,Y_test)
    table_80_70_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_70_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.9506374108025764
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False      31863    29    31892
True        1818  3707     5525
__all__    33681  3736    37417

Accuracy of the model for test data is: 0.9410947188368612
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       7969    16     7985
True         535   834     1369
__all__     8504   850     9354



Logistic Regression
Accuracy of the model for training data is: 0.9552609776304888
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       7880   105     7985
True         380   989     1369
__all__     8260  1094     9354



Neural Network-1
The accuracy of the model on training data is: 0.9979955635139108


Predicted  False  True  __all__
Actual                         
False      31854    38    31892
True          37  5488     5525
__all__    31891  5526    37417

The accuracy of the model on test data is: 0.9537096429335044


Predicted  False  True  __all__
Actual                         
False       7800   185     7985
True         248  1121     1369
__all__     8048  1306     9354

### <a id='5.A' style="text-decoration : none; color : #000000;"> A. Results</a>

In [89]:
trace0 = go.Scatter(x=table_80_70_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_70_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_70_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_70_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_70_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_70_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_70_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_70_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [90]:
table_80_70_train['index']=ind_columns
table_80_70_test['index']=ind_columns


table_80_70_test.set_index(['index'],drop=True,inplace=True)
table_80_70_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_70_train)
print("Test Results")
display(table_80_70_test)

Training Results


accuracy      TP       TN     FP      FN  \
index                                                                  
KNeighborsClassifier        0.950637  3707.0  31863.0   29.0  1818.0   
LogisticRegression          0.955261  4267.0  31476.0  416.0  1258.0   
RandomForestClassifier      0.994040  5307.0  31887.0    5.0   218.0   
AdaBoostClassifier          0.942753  3978.0  31297.0  595.0  1547.0   
GradientBoostingClassifier  0.956571  4193.0  31599.0  293.0  1332.0   
ExtraTreeClassifier         0.999973  5524.0  31892.0    0.0     1.0   
BaggingClassifier           0.993853  5301.0  31886.0    6.0   224.0   
Neural Network-1            0.997996  5488.0  31854.0   38.0    37.0   

                                 FPR       FNR  
index                                           
KNeighborsClassifier        0.000909  0.329050  
LogisticRegression          0.013044  0.227692  
RandomForestClassifier      0.000157  0.039457  
AdaBoostClassifier          0.018657  0.280000  
GradientBoostingClassifier  0.009187  0.241086  
ExtraTreeClassifier         0.000000  0.000181  
BaggingClassifier           0.000188  0.040543  
Neural Network-1            0.001192  0.006697

Test Results


accuracy      TP      TN     FP     FN       FPR  \
index                                                                          
KNeighborsClassifier        0.941095   834.0  7969.0   16.0  535.0  0.002004   
LogisticRegression          0.954993  1060.0  7873.0  112.0  309.0  0.014026   
RandomForestClassifier      0.943767   933.0  7895.0   90.0  436.0  0.011271   
AdaBoostClassifier          0.941736   982.0  7827.0  158.0  387.0  0.019787   
GradientBoostingClassifier  0.952320  1022.0  7886.0   99.0  347.0  0.012398   
ExtraTreeClassifier         0.940560   857.0  7941.0   44.0  512.0  0.005510   
BaggingClassifier           0.948151   989.0  7880.0  105.0  380.0  0.013150   
Neural Network-1            0.953710  1121.0  7800.0  185.0  248.0  0.023168   

                                 FNR       auc       mcc  
index                                                     
KNeighborsClassifier        0.390796  0.803600  0.746706  
LogisticRegression          0.225712  0.880131  0.811724  
RandomForestClassifier      0.318481  0.835124  0.759082  
AdaBoostClassifier          0.282688  0.848762  0.753649  
GradientBoostingClassifier  0.253470  0.867066  0.798972  
ExtraTreeClassifier         0.373996  0.810247  0.743377  
BaggingClassifier           0.277575  0.854638  0.780109  
Neural Network-1            0.181154  0.897839  0.811448

In [91]:
### Original Creator : Darshan Bhansali
### HTML code to hide the input cells 
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle code, click <a href="javascript:code_toggle()">here</a>.''')